# Slack Alerting - Suspicious User Login

This example demonstrates how to use GlassFlow to send a Slack message if there is a user with suspicious login

## Pre-requisites

- Create your free GlassFlow account via the [GlassFlow WebApp](https://app.glassflow.dev).
- Get your [Personal Access Token](https://app.glassflow.dev/profile) to authorize the Python SDK to interact with GlassFlow Cloud.
- Slack Incoming Webhook URL (https://api.slack.com/messaging/webhooks).
- Get your Geoapify API key from https://myprojects.geoapify.com/ . This key is used in the transformation stage to get geo location data



In [ ]:
%pip install "glassflow>=2.0.8" pandas Faker

In [ ]:
import glassflow

In [ ]:
# fill credentials
# Please edit this variable with your own personal access token from https://app.glassflow.dev/profile
PERSONAL_ACCESS_TOKEN = ""
GEOAPI_KEY = ""
SLACK_WEBHOOK_URL = ""
SLACK_WEBHOOK_HEADERS = [{"name": "Content-type", "value": "application/json"}]

## Create Pipeline

In [ ]:
client = glassflow.GlassFlowClient(
    personal_access_token=PERSONAL_ACCESS_TOKEN
)

In [ ]:
# Get the space named "examples" (or create one if no space is found)
list_spaces = client.list_spaces()

space_name = "examples"
for s in list_spaces.spaces:
    if s["name"] == space_name:
        space = glassflow.Space(
            personal_access_token=client.personal_access_token,
            id=s["id"], 
            name=s["name"]
        )
        break
else:
    space = client.create_space(name=space_name)

print(f"Space \"{space.name}\" with ID: {space.id}")

### Transformation Function

In [ ]:
%pycat transform.py

### Env Variables needed for transformation

In [ ]:
env_vars = [{
  "name": "GEOAPI_KEY",
  "value": GEOAPI_KEY
}]

### Requirements txt

In [ ]:
with open("requirements.txt") as f:
    requirements_txt = f.read()
display(requirements_txt)

### Create Pipeline

In [ ]:
pipeline_name = "slack-alerting-example"

pipeline = client.create_pipeline(
    name=pipeline_name, 
    transformation_file='transform.py',
    space_id=space.id, 
    env_vars=env_vars, 
    requirements=requirements_txt,
    sink_kind="webhook",
    sink_config={
        "url": SLACK_WEBHOOK_URL,
        "method": "POST",
        "headers": SLACK_WEBHOOK_HEADERS
    }
)
print("Pipeline ID:", pipeline.id)

## Produce data and send it to your pipeline

### Create a dummy data generator using python faker library

In [ ]:
from faker import Faker

def user_login_generator():
    fake = Faker()
    return {
        'event_name': "user_login",
        'username': fake.name(),
        'email': fake.email(),
        'ip': fake.ipv4(),
        'known_ip': fake.boolean()
    }

In [ ]:
### Get pipeline data source object to publish events to the pipeline
data_source = pipeline.get_source()

In [ ]:
# Generate some data and send it to the pipeline. Store it locally to compare
n_events = 10
input_events = []
for i in range(n_events):
    event = user_login_generator()
    input_events.append(event)
    data_source.publish(event)

In [ ]:
## Display data sent to the pipeline
import pandas as pd

display(pd.DataFrame(input_events))

## Explore the pipeline on the web-UI


In [ ]:
pipeline_url = f"https://app.glassflow.dev/pipelines/{pipeline.id}"
print(pipeline_url)